In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName('spark_test_notes').getOrCreate()
import os
print(os.listdir("../input"))

In [ ]:
df = spark.read.csv("../input/metro-bike-share-trip-data.csv",inferSchema=True,header=True)

In [ ]:
summary_df=df.describe()

In [ ]:
summary_df.toPandas()

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import desc
df.groupBy("Trip ID").count().sort(desc("count")).show(5)

In [ ]:
df.select("Duration").describe().show()

In [ ]:
from pyspark.sql.functions import desc
df.groupBy("Starting Station ID").count().sort(desc("count")).show(5)

In [ ]:
from pyspark.sql.functions import desc
df.groupBy("Ending Station ID").count().sort(desc("count")).show(5)

In [ ]:
from pyspark.sql.functions import desc
df.groupBy("Bike ID").count().sort(desc("count")).show(5)
df.groupBy("Plan Duration").count().sort(desc("count")).show()

In [ ]:
from pyspark.sql.functions import desc
df.groupBy("Trip Route Category").count().sort(desc("count")).show()
df.groupBy("Passholder Type").count().sort(desc("count")).show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
PT = df.groupBy("Passholder Type").count().sort(desc("count"))
PT = PT.toPandas()
sns.barplot(x="Passholder Type",y="count",data=PT)
plt.title("Distribution by Passholder Type")

In [ ]:
sub_df = df.select("Trip ID","Starting Station ID","Ending Station ID")

In [ ]:
sub_df.head(5)

In [ ]:
sub_df.select("Trip ID").count()

In [ ]:
sub_df = sub_df.dropna()
sub_df.count()

In [ ]:
sub_df.groupBy(["Starting Station ID","Ending Station ID"]).count().sort(desc("count")).show()

In [ ]:
sub_df =sub_df.filter(sub_df["Starting Station ID"]!=sub_df["Ending Station ID"])

In [ ]:
sub_df = sub_df.dropDuplicates(["Starting Station ID","Ending Station ID"])
sub_df.count()

In [ ]:
sub_df.filter(sub_df["Starting Station ID"]==sub_df["Ending Station ID"]).count()

In [ ]:
sub_df.filter(sub_df["Starting Station ID"].between(3000,3005)).show()

In [ ]:
sub_df = sub_df.withColumnRenamed("Trip ID","id")
sub_df.head(5)

In [ ]:
from pyspark.sql.functions import *
sub_df = sub_df.withColumn("items",array("Starting Station ID","Ending Station ID"))

In [ ]:
sub_df.show(5)
sub_df.printSchema()

In [ ]:
sub_df = sub_df.select("id","items")
sub_df.show(5)
sub_df.printSchema()
sub_df.count()

In [ ]:
sub_df = sub_df.dropDuplicates(["items"])
sub_df.count()

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.0001, minConfidence=0.0001)
model = fpGrowth.fit(sub_df)

In [ ]:
model.freqItemsets.sort(desc("freq")).show()

In [ ]:
model.associationRules.sort(desc("confidence")).show(25)

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
idf = load_iris()

In [ ]:
features = pd.DataFrame(idf["data"])
labels = pd.DataFrame(idf["target"])

In [ ]:
features.columns =(["Sepal_Lenght","Sepal_Width","Petal_Length","Petal_Width"])
labels.columns = (["label"])

In [ ]:
iris_df = pd.concat([features,labels],axis=1)
iris_df.tail()

In [ ]:
iris = spark.createDataFrame(iris_df)

In [ ]:
names = iris.drop('label').columns

In [ ]:
iris.columns

In [ ]:
iris.show(10)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [ ]:
assembler = VectorAssembler(inputCols=["Sepal_Lenght","Sepal_Width","Petal_Length","Petal_Width"],outputCol="features")

In [ ]:
iris = assembler.transform(iris)

In [ ]:
iris = iris.select("label","features")
iris.show(5)

In [ ]:
(training,testing) = iris.randomSplit([0.7, 0.3],seed=1234)
training
testing

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
dtc = RandomForestClassifier(featuresCol="features",labelCol="label",seed=1234,numTrees=25)

In [ ]:
pipeline = Pipeline(stages=[dtc])

In [ ]:
dtc_model = pipeline.fit(training)

In [ ]:
dtc_predictions = dtc_model.transform(testing)

In [ ]:
dtc_predictions.show(5)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(dtc_predictions)
print("Accuracy: ",accuracy)
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
dtc_predictions.groupBy("label").pivot("prediction").count().show()